In [27]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt
import gradio as gr

from numpy import asarray
from numpy import zeros

import re
import json
import locale
import platform
from datetime import datetime
import numpy

if platform.system() == 'Linux':
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
elif platform.system() == 'Windows':
    locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/enriquegomeztagle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# Lectura de formatos .json para entrenar cada modelo y asignación
# de información correspondiente
with open('Intenciones_NivelI.json', encoding='utf-8') as file:
    data_NivelI = json.load(file)

with open('Intenciones_NivelIIA.json', encoding='utf-8') as file:
    data_NivelIIA = json.load(file)

with open('Intenciones_NivelIIB.json', encoding='utf-8') as file:
    data_NivelIIB = json.load(file)


In [29]:
NI = dict()
NIIA = dict()
NIIB = dict()

for info in data_NivelI['intents']:
    NI.setdefault(info['tag'], info['patterns'])

for info in data_NivelIIA['intents']:
    NIIA.setdefault(info['tag'], info['patterns'])

for info in data_NivelIIB['intents']:
    NIIB.setdefault(info['tag'], info['patterns'])

# print(NI)
# print(NIIA)
# print(NIIB)


In [30]:
Y_NI = list()
Y_NIIA = list()
Y_NIIB = list()

for clase, lista_textos in NI.items():
    for text in lista_textos:
        Y_NI.append(list(NI.keys()).index(clase))

for clase, lista_textos in NIIA.items():
    for text in lista_textos:
        Y_NIIA.append(list(NIIA.keys()).index(clase))

for clase, lista_textos in NIIB.items():
    for text in lista_textos:
        Y_NIIB.append(list(NIIB.keys()).index(clase))

# print("Vector de salidas Y para N1:")
# print(Y_NI)
# print("Vector de salidas Y para N2A:")
# print(Y_NIIA)
# print("Vector de salidas Y para N2B:")
# print(Y_NIIB)


In [31]:
def quitar_stopwords(Textos):
    X = list()
    for sentence in Textos:
        for stopword in stop_words:
            sentence = sentence.replace(" " + stopword + " ", " ")
        sentence = sentence.replace("á", "a")
        sentence = sentence.replace("é", "e")
        sentence = sentence.replace("í", "i")
        sentence = sentence.replace("ó", "o")
        sentence = sentence.replace("ú", "u")

        sentence = re.sub(r'\s+', ' ', sentence)
        sentence = sentence.lower()
        tokenizer = RegexpTokenizer(r'\w+')
        result = tokenizer.tokenize(sentence)
        X.append(TreebankWordDetokenizer().detokenize(result))
    return X


In [32]:
Textos_NI = list()
for Lista in NI.values():
    for Texto in Lista:
        Textos_NI.append(Texto)

X_NI = quitar_stopwords(Textos_NI)

Textos_NIIA = list()
for Lista in NIIA.values():
    for Texto in Lista:
        Textos_NIIA.append(Texto)

X_NIIA = quitar_stopwords(Textos_NIIA)

Textos_NIIB = list()
for Lista in NIIB.values():
    for Texto in Lista:
        Textos_NIIB.append(Texto)

X_NIIB = quitar_stopwords(Textos_NIIB)


In [33]:
# print(X_NI)
# print(X_NIIA)
# print(X_NIIB)


In [34]:
maxlen = 5

tokenizer_NI = Tokenizer(num_words=5000)
tokenizer_NIIA = Tokenizer(num_words=5000)
tokenizer_NIIB = Tokenizer(num_words=5000)

tokenizer_NI.fit_on_texts(X_NI)
X_NI_Tok = tokenizer_NI.texts_to_sequences(X_NI)
tokenizer_NIIA.fit_on_texts(X_NIIA)
X_NIIA_Tok = tokenizer_NIIA.texts_to_sequences(X_NIIA)
tokenizer_NIIB.fit_on_texts(X_NIIB)
X_NIIB_Tok = tokenizer_NIIB.texts_to_sequences(X_NIIB)


X_NI_train = pad_sequences(X_NI_Tok, padding='post', maxlen=maxlen)
X_NIIA_train = pad_sequences(X_NIIA_Tok, padding='post', maxlen=maxlen)
X_NIIB_train = pad_sequences(X_NIIB_Tok, padding='post', maxlen=maxlen)


In [35]:
# print("Matriz de entrada para NI:")
# print(X_NI_train)


In [36]:
# print("Matriz de entrada para NIIA:")
# print(X_NIIA_train)


In [37]:
# print("Matriz de entrada para NIIB:")
# print(X_NIIB_train)


In [38]:
embeddings_dictionary = dict()
Embeddings_file = open('Word2Vect_Spanish.txt', encoding="utf8")

for linea in Embeddings_file:
    caracts = linea.split()
    palabra = caracts[0]
    vector = asarray(caracts[1:], dtype='float32')
    embeddings_dictionary[palabra] = vector
Embeddings_file.close()


In [39]:
def Asignar_Embeddings(tokenizer, vocab_size):
    embedding_matrix = zeros((vocab_size, 300))
    for word, index in tokenizer.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    return embedding_matrix


In [40]:
vocab_size_NI = len(tokenizer_NI.word_index) + 1
embedding_matrix_NI = Asignar_Embeddings(tokenizer_NI, vocab_size_NI)

vocab_size_NIIA = len(tokenizer_NIIA.word_index) + 1
embedding_matrix_NIIA = Asignar_Embeddings(tokenizer_NIIA, vocab_size_NIIA)

vocab_size_NIIB = len(tokenizer_NIIB.word_index) + 1
embedding_matrix_NIIB = Asignar_Embeddings(tokenizer_NIIB, vocab_size_NIIB)


In [41]:
def Definir_Modelos(vocab_size, embedding_matrix, X_train, labels):
    model = Sequential()
    embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=X_train.shape[1],
                                trainable=False)
    model.add(embedding_layer)
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(len(labels), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # print(model.summary())

    # print("\nPalabras en el vocabulario:")
    # print(vocab_size)
    return model


In [42]:
model_NI = Definir_Modelos(vocab_size_NI, embedding_matrix_NI, X_NI_train, NI.keys())


In [43]:
model_NIIA = Definir_Modelos(vocab_size_NIIA, embedding_matrix_NIIA, X_NIIA_train, NIIA.keys())


In [44]:
model_NIIB = Definir_Modelos(vocab_size_NIIB, embedding_matrix_NIIB, X_NIIB_train, NIIB.keys())


In [45]:
def Entrenar_Modelos(X_train, Y, model, labels):
    train_labels = to_categorical(Y, num_classes=len(labels))
    # print('Matriz de salidas')
    # print(train_labels)

    history = model.fit(X_train, train_labels, epochs=30, batch_size=1, verbose=1)

    score = model.evaluate(X_train, train_labels, verbose=1)
    # print("\nTest Loss:", score[0])
    # print("Test Accuracy:", score[1])
    return history


In [46]:
history_NI = Entrenar_Modelos(X_NI_train, Y_NI, model_NI, NI.keys())


Epoch 1/30


2023-11-06 11:26:05.130998: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


62/62 [==============================] - 2s 7ms/step - loss: 1.2334 - accuracy: 0.3387
Epoch 2/30
62/62 [==============================] - 0s 7ms/step - loss: 0.7489 - accuracy: 0.7419
Epoch 3/30
62/62 [==============================] - 0s 5ms/step - loss: 0.2971 - accuracy: 1.0000
Epoch 4/30
62/62 [==============================] - 0s 5ms/step - loss: 0.1106 - accuracy: 0.9677
Epoch 5/30
62/62 [==============================] - 0s 5ms/step - loss: 0.1933 - accuracy: 0.9194
Epoch 6/30
62/62 [==============================] - 0s 5ms/step - loss: 0.0376 - accuracy: 0.9839
Epoch 7/30
62/62 [==============================] - 0s 5ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 8/30
62/62 [==============================] - 0s 5ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 9/30
62/62 [==============================] - 0s 6ms/step - loss: 0.0147 - accuracy: 1.0000
Epoch 10/30
62/62 [==============================] - 0s 5ms/step - loss: 0.0205 - accuracy: 1.0000
Epoch 11/30
62/62 [===========

In [47]:
history_NIIA = Entrenar_Modelos(X_NIIA_train, Y_NIIA, model_NIIA, NIIA.keys())


Epoch 1/30
51/51 [==============================] - 1s 4ms/step - loss: 1.3105 - accuracy: 0.5294
Epoch 2/30
51/51 [==============================] - 0s 5ms/step - loss: 0.6825 - accuracy: 0.7843
Epoch 3/30
51/51 [==============================] - 0s 5ms/step - loss: 0.2493 - accuracy: 0.9020
Epoch 4/30
51/51 [==============================] - 0s 5ms/step - loss: 0.0558 - accuracy: 1.0000
Epoch 5/30
51/51 [==============================] - 0s 5ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 6/30
51/51 [==============================] - 0s 5ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 7/30
51/51 [==============================] - 0s 5ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 8/30
51/51 [==============================] - 0s 5ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 9/30
51/51 [==============================] - 0s 5ms/step - loss: 0.1839 - accuracy: 0.9412
Epoch 10/30
51/51 [==============================] - 0s 5ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 11/30
51/51 [

In [48]:
history_NIIB = Entrenar_Modelos(X_NIIB_train, Y_NIIB, model_NIIB, NIIB.keys())


Epoch 1/30
25/25 [==============================] - 1s 4ms/step - loss: 1.0987 - accuracy: 0.4000
Epoch 2/30
25/25 [==============================] - 0s 6ms/step - loss: 0.9749 - accuracy: 0.7200
Epoch 3/30
25/25 [==============================] - 0s 5ms/step - loss: 0.7358 - accuracy: 0.8400
Epoch 4/30
25/25 [==============================] - 0s 5ms/step - loss: 0.3475 - accuracy: 0.8400
Epoch 5/30
25/25 [==============================] - 0s 5ms/step - loss: 0.1045 - accuracy: 0.9600
Epoch 6/30
25/25 [==============================] - 0s 5ms/step - loss: 0.0712 - accuracy: 1.0000
Epoch 7/30
25/25 [==============================] - 0s 5ms/step - loss: 0.1227 - accuracy: 0.9600
Epoch 8/30
25/25 [==============================] - 0s 5ms/step - loss: 0.2839 - accuracy: 0.9200
Epoch 9/30
25/25 [==============================] - 0s 5ms/step - loss: 0.0845 - accuracy: 0.9600
Epoch 10/30
25/25 [==============================] - 0s 5ms/step - loss: 0.0399 - accuracy: 1.0000
Epoch 11/30
25/25 [

In [49]:
def Grafica_Modelo(history):
    plt.figure(figsize=(12, 5))
    plt.ylim(-0.1, 1.1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Acc', 'Loss'])
    plt.show()


In [50]:
# Grafica_Modelo(history_NI)


In [51]:
# Grafica_Modelo(history_NIIA)


In [52]:
# Grafica_Modelo(history_NIIB)


# Programación del Árbol conversacional

In [53]:
def instancer(inp, model, tags):
    inp = inp.lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o")
    inp = inp.replace("ú", "u").replace("¿", "").replace("?", "")
    txt = [inp]
    seq = tokenizer_NI.texts_to_sequences(txt)
    padded = pad_sequences(seq, maxlen=maxlen)

    # print("Input shape before predict:", padded.shape)
    # print("Input data:", padded)

    results = model.predict(padded)
    results_index = numpy.argmax(results)
    tag = list(tags.keys())[results_index]
    maxscore = numpy.max(results)
    return tag, maxscore


In [54]:

def Activar_NI(user_input):
    if user_input is None:
        response = "ChatBot: ¿En qué puedo ayudarte? Puedes preguntarme sobre reservaciones, servicios del hotel, su historia o atracciones cercanas."
        return response, 'NI'

    tag, maxscore = instancer(user_input, model_NI, NI)

    if maxscore > 0.65 or user_input == 'salir':
        if user_input == 'salir':
            response = "ChatBot: Hasta luego, fue un gusto hablar contigo"
            return response, 'salir'

        if tag == 'Reservar_Habitacion':
            if user_input.count('Tipo_Habitacion') > 0:
                next_state = 'NIIA1'
            elif user_input.count('Fecha_entrada') > 0:
                next_state = 'NIIA2'
            elif user_input.count('Fecha_salida') > 0:
                next_state = 'NIIA3'
            elif user_input.count("Num_huespedes") > 0:
                next_state = 'NIIA4'
            else:
                next_state = 'NIIA'
            response = "ChatBot: Entendido, estás interesado en reservar una habitación."

        elif tag == 'Servicio_hotel':
            if user_input.count('Apartar_espacio') > 0:
                next_state = 'NIIB1'
            elif user_input.count('Servicio') > 0:
                next_state = 'NIIB2'
            elif user_input.count('Comida_bebida') > 0:
                next_state = 'NIIB3'
            else:
                next_state = 'NIIB'
            response = "ChatBot: Claro, puedo ayudarte con los servicios del hotel."

        elif tag == 'Historia_Hotel':
            response = "ChatBot: Desde su apertura en 1920 por el pionero Don Eduardo Mendoza, nuestro hotel es un ícono de la arquitectura Art Deco, diseñado por Carlos Fontana.\nHa acogido a luminarias como la actriz Clara Estrella en los años 30 y fue el lugar de la histórica cumbre de paz en 1955. Cada habitación cuenta una historia; por ejemplo, la suite 204, donde el novelista Luis Montero escribió su obra maestra. Es un lugar donde la historia y la hospitalidad se encuentran en cada esquina."
            next_state = 'NI'

        elif tag == 'Atracciones_Cercanas':
            response = "Chatbot: En el corazón de la ciudad, nuestro hotel se encuentra a un corto trayecto del Parque Central y a unos minutos del Museo Nacional de Arte, perfecto para los entusiastas de la cultura. Para aquellos interesados en la historia, el Convento de San Francisco está a una caminata de distancia. No te pierdas el Mercado del Sol, ideal para saborear la cocina local y encontrar artesanías únicas. Además, ofrecemos excursiones al Castillo de San Lorenzo, un sitio histórico con vistas espectaculares, a solo media hora en coche."
            next_state = 'NI'

        else:
            next_state = 'NI'
            response = "ChatBot: Puedes preguntarme sobre reservaciones, servicios del hotel, su historia o atracciones cercanas."

        return response, next_state

    else:
        response = "ChatBot: Lo siento, pero no entendí tu petición, ¿Podrías decirlo de otra forma?"
        return response, 'NI'


In [55]:
def Activar_NIIA(user_input):
    if user_input is None:
        response = "ChatBot: Entiendo que quieres reservar una habitación. Por favor, dime 'Reservar Habitación' para iniciar el proceso."
        return response, 'NIIA'

    tag, maxscore = instancer(user_input, model_NIIA, NIIA)

    if maxscore > 0.5 or user_input == 'salir' or user_input == 'volver':
        if user_input == 'volver':
            next_state = 'NI'
            response = "ChatBot: Entendido, volviendo al menú principal."
        elif user_input == 'salir':
            response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
            return response

        elif tag == 'Tipo_Habitacion' or not Tipo_Cuarto:
            next_state = 'NIIA1'
            response = "Sigamos con el proceso"
        elif tag == 'Fecha_entrada':
            next_state = 'NIIA2'
            response = "Sigamos con el proceso"
        elif tag == 'Fecha_salida':
            next_state = 'NIIA3'
            response = "Sigamos con el proceso"
        elif tag == 'Num_huespedes':
            next_state = 'NIIA4'
            response = "Sigamos con el proceso"
        else:
            next_state = 'NIIA'
            response = "ChatBot: Por favor, dime más sobre tu reserva."

        return response, next_state
    else:
        response = "ChatBot: Lo siento, pero no entendí. Puedes decirme 'Reservar Habitación' por ejemplo."
        return response, 'NIIA'


In [56]:
def Activar_NIIA1(user_input=None):
    if user_input is None:
        response = "ChatBot: Excelente! Reservemos una habitación. ¿Qué tipo de habitación te gustaría reservar?"
        return response, 'NIIA1'
    
    user_input = user_input.lower()

    if 'estándar' in user_input or 'económica' in user_input:
        response = "ChatBot: Excelente, reservaremos una habitación estándar."
        Tipo_Cuarto = "Estándar"
        next_state = 'NIIA2'
    elif 'suite' in user_input:
        response = "ChatBot: Excelente, reservaremos una habitación suite."
        Tipo_Cuarto = "Suite"
        next_state = 'NIIA2'
    elif 'doble' in user_input or 'dos camas' in user_input:
        response = "ChatBot: Excelente, reservaremos una habitación doble."
        Tipo_Cuarto = "Doble"
        next_state = 'NIIA2'
    elif 'familiar' in user_input:
        response = "ChatBot: Excelente, reservaremos una habitación familiar."
        Tipo_Cuarto = "Familiar"
        next_state = 'NIIA2'
    elif 'deluxe' in user_input:
        response = "ChatBot: Excelente, reservaremos una habitación deluxe."
        Tipo_Cuarto = "Deluxe"
        next_state = 'NIIA2'
    elif 'ofrecen' in user_input:
        response = "ChatBot: Ofrecemos los siguientes tipos de habitaciones: estándar, doble, familiar, suite, y deluxe."
    elif user_input == 'volver':
        next_state = 'NIIA'
        response = "ChatBot: De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        next_state = 'salir'
        response = "ChatBot: Hasta luego, fue un gusto ayudarte."
    else:
        next_state = 'NIIA1'
        response = "ChatBot: No he podido identificar el tipo de habitación. ¿Puedes especificar si deseas una habitación estándar, suite, doble, etc.?"

    return response, next_state


In [57]:
def Activar_NIIA2(user_input):
    if user_input is None:
        response = "ChatBot: Muy bien! ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
        return response, 'NIIA2'

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "ChatBot: De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    else:
        try:
            fecha_entrada_usuario = datetime.strptime(user_input, '%d/%m/%Y')
            if fecha_entrada_usuario.date() < datetime.now().date():
                response = "ChatBot: No puedes agendar una fecha en el pasado. Por favor, elige una fecha futura."
                next_state = 'NIIA2'
            else:
                fecha_formateada = fecha_entrada_usuario.strftime('%d de %B del %Y')
                response = f"ChatBot: Muy bien, agendaré la entrada para el {fecha_formateada}."
                Fecha_entrada = fecha_entrada_usuario
                next_state = 'NIIA3'
        except ValueError:
            response = "ChatBot: Parece que la fecha no está en el formato correcto. Por favor, ingresa la fecha en formato dd/mm/aaaa."
            next_state = 'NIIA2'

    return response, next_state


In [58]:
def Activar_NIIA3(user_input):
    if user_input is None:
        response = "ChatBot: Muy bien! ¿Para cuándo agendamos su salida? (dd/mm/aaaa)"
        return response, 'NIIA3'

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "ChatBot: De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    else:
        try:
            fecha_salida_usuario = datetime.strptime(user_input, '%d/%m/%Y')
            if fecha_salida_usuario.date() < datetime.now().date():
                response = "ChatBot: No puedes agendar una fecha de salida en el pasado. Por favor, elige una fecha futura."
            elif Fecha_entrada and fecha_salida_usuario.date() <= Fecha_entrada.date():
                response = "ChatBot: La fecha de salida debe ser posterior a la fecha de entrada. Por favor, elige una fecha adecuada."
                next_state = 'NIIA3'
            else:
                fecha_formateada = fecha_salida_usuario.strftime('%d de %B del %Y')
                response = f"ChatBot: Muy bien, agendaré la salida para el {fecha_formateada}."
                Fecha_salida = fecha_salida_usuario
                next_state = 'NIIA4'
        except ValueError:
            response = "ChatBot: Parece que la fecha no está en el formato correcto. Por favor, ingresa la fecha en formato dd/mm/aaaa."
            next_state = 'NIIA3'

    return response, next_state


In [59]:
def Activar_NIIA4(user_input):
    if user_input is None:
        response = "ChatBot: Súper! ¿Para cuántas personas es la reservación?"
        return response, 'NIIA4'

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "ChatBot: De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    else:
        try:
            huespedes = int(user_input)
            if 1 <= huespedes <= 4:
                response = f"ChatBot: Muy bien, agendaré para {huespedes} personas.\n" \
                           f"La reservación final es de {huespedes} personas con el tipo de cuarto {Tipo_Cuarto}, " \
                           f"con fecha de entrada {Fecha_entrada.strftime('%d/%m/%Y')}, " \
                           f"y fecha de salida {Fecha_salida.strftime('%d/%m/%Y')}. " \
                           f"\n\nSi deseas algo más no dudes en preguntar."
                Num_huespedes = huespedes
                next_state = 'NI'
            else:
                response = "ChatBot: Actualmente solo podemos reservar para hasta 4 personas."
                next_state = 'NIIA4'
        except ValueError:
            response = "ChatBot: ¿Podrías indicarme un número de personas para la reservación?"
            next_state = 'NIIA4'

    return response, next_state


In [60]:
def Activar_NIIB(user_input=None):
    if user_input is None:
        response = "ChatBot: Entiendo que quieres pedir un servicio del Hotel. " \
                   "(Apartar espacio, Pedir servicio al cuarto, Pedir comida o bebida)"
        return response, 'NIIB'

    tag, maxscore = instancer(user_input, model_NIIB, NIIB)

    if maxscore > 0.5 or user_input == 'salir' or user_input == 'volver':
        if user_input == 'volver':
            next_state = 'NI'
            response = "ChatBot: De acuerdo, volviendo al menú anterior."
        elif user_input == 'salir':
            response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
            next_state = 'salir'
        elif tag == 'Apartar_espacio':
            next_state = 'NIIB1'
            response = "Sigamos con el proceso"
        elif tag == 'Servicio':
            next_state = 'NIIB2'
            response = "Sigamos con el proceso"
        elif tag == 'Comida_bebida':
            next_state = 'NIIB3'
            response = "Sigamos con el proceso"
        else:
            response = "ChatBot: No estoy seguro de cómo ayudarte con eso. ¿Podrías especificar más tu solicitud?"
            next_state = 'NIIB'
    else:
        response = "ChatBot: Lo siento, pero no entendí. ¿Podrías ser más específico?"
        next_state = 'NIIB'

    return response, next_state


In [61]:
def Activar_NIIB1(user_input):
    if user_input is None:
        response = "ChatBot: Excelente! Apartemos un espacio. ¿Qué tipo de espacio te gustaría reservar?"
        next_state = 'NIIB1'
        return response, next_state

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "ChatBot: De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    elif 'alberca' in user_input:
        response = "ChatBot: Excelente, reservaremos la alberca. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "alberca"
        next_state = 'NI'
    elif 'gimnasio' in user_input:
        response = "ChatBot: Excelente, reservaremos el gimnasio. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "gimnasio"
        next_state = 'NI'
    elif 'eventos' in user_input or 'salon' in user_input:
        response = "ChatBot: Excelente, reservaremos el salón de eventos. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "salón de eventos"
        next_state = 'NI'
    elif 'terraza' in user_input:
        response = "ChatBot: Excelente, reservaremos la terraza. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "terraza"
        next_state = 'NI'
    elif 'spa' in user_input:
        response = "ChatBot: Excelente, reservaremos el spa. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "spa"
        next_state = 'NI'
    else:
        response = "ChatBot: ¿Podrías repetirme qué tipo de espacio te gustaría reservar?"
        next_state = 'NIIB1'

    return response, next_state


In [62]:
def Activar_NIIB2(user_input=None):
    if user_input is None:
        response = "ChatBot: Con gusto! ¿Qué servicio le gustaría pedir?"
        next_state = 'NIIB2'
        return response, next_state

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "ChatBot: De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    elif 'limpiar' in user_input:
        response = "ChatBot: Claro, enviaremos a alguien para limpiar la habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "limpieza de cuarto"
        next_state = 'NI'
    elif 'toallas' in user_input:
        response = "ChatBot: Ok, enviaremos toallas a su habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "toallas"
        next_state = 'NI'
    elif 'jabón' in user_input or 'shampoo' in user_input:
        response = "ChatBot: Seguro! enviaremos artículos de higiene personal a su cuarto. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "higiene personal"
        next_state = 'NI'
    elif 'equipaje' in user_input:
        response = "ChatBot: Mandaremos a alguien para apoyar con el equipaje. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "apoyo con equipaje"
        next_state = 'NI'
    elif 'almohadas' in user_input:
        response = "ChatBot: Con gusto enviaremos más almohadas a la habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "almohadas"
        next_state = 'NI'
    else:
        response = "ChatBot: ¿Podrías repetirme qué servicio requieres?"
        next_state = 'NIIB2'

    return response, next_state


In [63]:
def Activar_NIIB3(user_input=None):
    if user_input is None:
        response = "ChatBot: Hora de comer! ¿Qué alimento o bebida le gustaría pedir?"
        next_state = 'NIIB3'
        return response, next_state

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "ChatBot: De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "ChatBot: Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    elif 'desayuno' in user_input:
        response = "ChatBot: Claro, enviaremos el desayuno a la habitación."
        comida_pedida = "desayuno"
        next_state = 'NI'
    elif 'vegana' in user_input or 'vegetariana' in user_input:
        response = "ChatBot: Por supuesto, prepararemos una opción vegana/vegetariana para usted."
        comida_pedida = "vegana/vegetariana"
        next_state = 'NI'
    elif 'comida' in user_input:
        response = "ChatBot: Mandaremos la comida a la habitación."
        comida_pedida = "comida"
        next_state = 'NI'
    elif 'botella' in user_input or 'vino' in user_input:
        response = "ChatBot: Seguro! enviaremos una botella de vino."
        comida_pedida = "vino"
        next_state = 'NI'
    elif 'café' in user_input:
        response = "ChatBot: Con gusto enviaremos café a su habitación."
        comida_pedida = "café"
        next_state = 'NI'
    elif 'romántica' in user_input:
        response = "ChatBot: Prepararemos una cena romántica para usted."
        comida_pedida = "romántica"
        next_state = 'NI'
    elif 'cena' in user_input:
        response = "ChatBot: Claro, mandaremos la cena a la habitación."
        comida_pedida = "cena"
        next_state = 'NI'
    else:
        response = "ChatBot: ¿Podrías repetirme qué alimento/bebida quieres pedir?"
        next_state = 'NIIB3'

    return response, next_state


In [66]:
# Implementación de casos correspondientes para cada nivel del ChatBot
# Nivel contextual inicial por defecto, el primero
estado_actual: 'NI'
Tipo_Cuarto: ""
Fecha_entrada: ""
Fecha_salida: ""
Num_huespedes: 0
Servicio_deseado: ""
Espacio_Apartado: ""
Servicio_Pedido: ""
Comida_Pedida: ""

maquina_estados = {'NI': Activar_NI,
                   "NIIA": Activar_NIIA,
                   "NIIA1": Activar_NIIA1,
                   "NIIA2": Activar_NIIA2,
                   "NIIA3": Activar_NIIA3,
                   "NIIA4": Activar_NIIA4,
                   "NIIB": Activar_NIIB,
                   "NIIB1": Activar_NIIB1,
                   "NIIB2": Activar_NIIB2,
                   "NIIB3": Activar_NIIB3
                   }

def adapted_chat1(user_input, Nivel):
    if user_input == "":
        response, next_state = maquina_estados[Nivel](None)
    else:
        response, next_state = maquina_estados[Nivel](user_input)
    return response, next_state

def gradio_chatbot_function(user_input, state='NI'):
    if user_input == "":
        response, next_state = adapted_chat1(None, state)
    else:
        response, next_state = adapted_chat1(user_input, state)
    return response, next_state

iface = gr.Interface(
    fn=gradio_chatbot_function,
    inputs=[gr.Textbox(lines=2, label="Tu mensaje", placeholder="Escribe tu mensaje aquí..."), "state"],
    outputs=[gr.Textbox(label="Respuesta del Chatbot", placeholder="Puedes preguntarme sobre reservaciones, servicios del hotel, su historia o atracciones cercanas"), "state"],
    title="'Hotel ESDAI Chatbot",
    description="Bienvenido a Hotel ESDAI Chatbot. ¿Qué deseas hacer?"
)

iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 301ms/step


In [65]:
# # Check the size of the tokenizer's word index
# print("Tokenizer word index size:", len(tokenizer_NI.word_index))
# 
# # After loading the model, check the input dimension of the embedding layer
# embedding_input_dim = model_NIIB.layers[0].get_config()['input_dim']
# print("Model's embedding input dimension:", embedding_input_dim)
